first we need to create the Compute in the so cluster is created and and then go to catolog in the panale and upload the csv file from  https://github.com/databricks/Spark-The-Definitive-Guide/blob/master/data/flight-data/csv/2015-summary.csv.

come to Workspace and create a notebook and rename it

# Reading Data

for reading the file in spark -  Dataframreader associate with sparksession is used. here we want spark to do the schema inference i.e spark will give a best guess of the schema of our dataframe. also specify that First rwo is header. 
- Read the CSV file into a Spark DataFrame.
- Infer the schema (data types of columns) automatically.

#### Reading the CSV File into a DataFrame

In [0]:
dbutils.fs.ls('/FileStore/tables')

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/2015_summary.csv', name='2015_summary.csv', size=7080, modificationTime=1741666214000),
 FileInfo(path='dbfs:/FileStore/tables/BigMart_Sales-1.csv', name='BigMart_Sales-1.csv', size=869537, modificationTime=1740727435000),
 FileInfo(path='dbfs:/FileStore/tables/BigMart_Sales.csv', name='BigMart_Sales.csv', size=869537, modificationTime=1740138394000),
 FileInfo(path='dbfs:/FileStore/tables/drivers.json', name='drivers.json', size=180812, modificationTime=1740139360000)]

In [0]:
df=spark.read.format('csv')\
    .option('inferschema', True)\
        .option('header',True)\
            .load('dbfs:/FileStore/tables/2015_summary.csv')
#When we read the data into a DataFrame, Spark does not immediately process it. It just creates a plan to process it later.

>  Another way for reading the file. directly mentioning csv in the code.

In [0]:
flightData2015 = spark\
    .read\
    .option("inferSchema", True)\
    .option("header", True)\
    .csv("dbfs:/FileStore/tables/2015_summary.csv")  
#When we read the data into a DataFrame, Spark does not immediately process it. It just creates a plan to process it later.


- **spark.read** → Creates a DataFrameReader to read data.
- **.option("inferSchema", "true")** → Enables schema inference, meaning Spark will automatically guess the data types of columns.
- **.option("header", "true")** → Specifies that the first row contains column names.
- **.csv("/data/flight-data/csv/2015-summary.csv")** → Reads the CSV file into a DataFrame


schema inference is important because it helps spark to  improve performance because Spark can use optimized operations for numbers, dates, etc. However, in production systems, it's recommended to manually define the schema for better accuracy and consistency.


In [0]:
df.show(5)  # It triggers execution, and we see the first 5 rows.



+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



### Sorting the data

In [0]:
df_sorted = df.sort('count')

#This does not modify df (because transformations in Spark are lazy).
#It returns a new DataFrame (df_sorted) with rows ordered by flight count.

In [0]:
df_sorted.show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



#### Understanding the Execution Plan (explain plan)

.explain() on a DataFrame in PySpark, it shows you how Spark plans to execute the query.\
**How to Read an Explain Plan**
- The plan is structured top to bottom.
- The top part represents the final operation that Spark will perform.
- The bottom part represents the starting point, which is the data source.



In [0]:
df.sort('count').explain() # run this code

""" Sort [count#202 ASC NULLS FIRST], true, 0 :   Spark sorts the data by the column count (ascending order).
                                                  NULL values appear first.
                                                  true, 0: true means sorting is global, 0 refers to sorting order."""
""" Exchange rangepartitioning(count#202 ASC NULLS FIRST, 200) : Exchange means data shuffling across partitions. 
    rangepartitioning(count#202 ASC NULLS FIRST, 200):Uses Range Partitioning (divides data into ranges based on count values). 200 partitions are created. Ensures that sorting is distributed efficiently.
FileScan csv :  Columns Read: DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count
                Batched: false → Row-based processing (not columnar format like Parquet).
                DataFilters: [] → No filters applied (full scan).
                Format: CSV → Reading data from a CSV file.
                Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2015_summary.csv]
                File is stored in Databricks DBFS.
                PartitionFilters: [] → No partition pruning applied.
                PushedFilters: [] → No pushdown filters applied.
"""



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#202 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#202 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=281]
      +- FileScan csv [DEST_COUNTRY_NAME#200,ORIGIN_COUNTRY_NAME#201,count#202] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


Out[36]: ' Exchange rangepartitioning(count#202 ASC NULLS FIRST, 200) : Exchange means data shuffling across partitions. \n    rangepartitioning(count#202 ASC NULLS FIRST, 200):Uses Range Partitioning (divides data into ranges based on count values). 200 partitions are created. Ensures that sorting is distributed efficiently.\nFileScan csv :  Columns Read: DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count\n                Batched: false → Row-based processing (not col

In Above by default Spark uses 200 shuffle partitions, which can be too many for small datasets. We can reduce this number for better performance.


### Changing Shuffle Partitions

- Too many partitions (e.g., 200) → High overhead
  - More partitions mean more shuffling and network communication, leading to performance issues.
  - Too few partitions (e.g., 1) → Inefficient parallelism
If you reduce it too much, Spark won’t utilize all the worker nodes efficiently.
- The best partition size depends on your dataset and cluster configuration.
A good practice is to experiment with different values and monitor performance.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "50")

# spark.sql.shuffle.partitions is a Spark configuration setting that controls the number of partitions used when shuffling data during Spark SQL operation

- Every transformation (like sort) does not immediately execute.
- Instead, Spark builds a logical execution plan, keeping track of what needs to be done.
- This is known as lineage – Spark knows exactly how to recompute any piece of data if needed.
- Spark is designed for fault tolerance.
- If a worker node fails, Spark can recompute the lost data using its lineage.


In [0]:
df.sort("count").take(5)

Out[51]: [Row(DEST_COUNTRY_NAME='Malta', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='Saint Vincent and the Grenadines', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Gibraltar', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1)]


### Take() and Collect ()
take(n)- Returns only n rows (efficient for large datasets)\
collect() - Returns all rows (not recommended for large datasets) - Argumnets cant be passed to collect

In [0]:
df.take(5)


Out[54]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [0]:
df.collect()

Out[56]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', cou

# Data Frame and SQL

The main point is that in Apache Spark, you can express the same data transformations in two ways:
- Using DataFrame API (in Python, R, Scala, or Java)
- Using SQL queries
Both approaches compile to the same underlying execution plan, meaning there's no performance difference between them. This gives you flexibility to use whichever approach you prefer.


> **How to use SQL queries on DataFrame**

First register your DataFrame as a temporary view\
then use SQL API and you can query the data\
and the query results are by default sore in _sql+'var_name' which can again used back by python to perform operations

In [0]:
df.createOrReplaceTempView("df") #register your DataFrame as a temporary view

In [0]:
%sql
select * from df  -- the query results are by default sore in _sql+'var_name' which can again used back by python to perform operations

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [0]:
_sqldf.display() #the query results are by default sore in _sql+'var_name' which can again used back by python to perform operations

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


### or instead of changing to SQL API we can use spark.sql Funtion in Python adn write SQL query in it

In [0]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(*)
FROM  df
GROUP BY DEST_COUNTRY_NAME
""")
sqlWay.display()

#SQL APPROACH

DEST_COUNTRY_NAME,count(1)
Panama,1
Cape Verde,1
Hong Kong,1
Canada,1
Anguilla,1
Russia,1
Paraguay,1
Venezuela,1
Finland,1
Ukraine,1


In [0]:
#md
DATAframe Way in Python-Pyspark

In [0]:
dataFrameWay = df.groupBy("DEST_COUNTRY_NAME").count()

dataFrameWay.display()

DEST_COUNTRY_NAME,count
Panama,1
Cape Verde,1
Hong Kong,1
Canada,1
Anguilla,1
Russia,1
Paraguay,1
Venezuela,1
Finland,1
Ukraine,1



From above it can be seen that both follow same appraoch by providing Identical results can be verified using Explain ()


In [0]:
sqlWay.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#200], functions=[finalmerge_count(merge count#553L) AS count(1)#546L])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#200, 50), ENSURE_REQUIREMENTS, [plan_id=961]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#200], functions=[partial_count(1) AS count#553L])
         +- FileScan csv [DEST_COUNTRY_NAME#200] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [0]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#200], functions=[finalmerge_count(merge count#569L) AS count(1)#562L])
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 0, Statistics(sizeInBytes=4.6 KiB, rowCount=132, isRuntime=true)
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#200, 50), ENSURE_REQUIREMENTS, [plan_id=804]
            +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#200], functions=[partial_count(1) AS count#569L])
               +- FileScan csv [DEST_COUNTRY_NAME#200] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
+- == Initial Plan ==
   HashAggregate(keys=[DEST_COUNTRY_NAME#200], functions=[finalmerge_count(merge count#569L) AS count(1)#562L])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#200, 50), ENSURE_REQUIREME

Directed Acyclic Graph (DAG): - > How Spark represents the sequence of transformations